## Day 17: Pyroclastic Flow ---

Your handheld device has located an alternative exit from the cave for you and the elephants. The ground is rumbling almost continuously now, but the strange valves bought you some time. It's definitely getting warmer in here, though.

The tunnels eventually open into a very tall, narrow chamber. Large, oddly-shaped rocks are falling into the chamber from above, presumably due to all the rumbling. If you can't work out where the rocks will fall next, you might be crushed!

In [1]:
import pytest
from icecream import ic

The five types of rocks have the following peculiar shapes, where # is rock and . is empty space:

```
####
```

```
.#.
###
.#.
```

```
..#
..#
###
```

```
#
#
#
#
```

```
##
##
```

The rocks fall in the order shown above: first the `-` shape, then the `+` shape, and so on. Once the end of the list is reached, the same order repeats: the `-` shape falls first, sixth, 11th, 16th, etc.

The rocks don't spin, but they do get pushed around by jets of hot gas coming out of the walls themselves. A quick scan reveals the effect the jets of hot gas will have on the rocks as they fall (your puzzle input).

For example, suppose this was the jet pattern in your cave:

```
>>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>
```

In [2]:
def load_input(filename):
    with open(filename, 'r') as f_input:
        for line in f_input:
            line = line.strip()
            for char in line:
                yield char
      
jets = list(load_input('sample.txt'))
assert len(jets) == 40
assert jets[0] == '>'
assert jets[3] == '<'
assert jets[-1] == '>'

In jet patterns, `<` means a push to the left, while `>` means a push to the right. The pattern above means that the jets will push a falling rock right, then right, then right, then left, then left, then right, and so on. If the end of the list is reached, it repeats.

The tall, vertical chamber is exactly **seven units wide**. Each rock appears so that its left edge is **two units away from the left wall** and its bottom edge is **three units above the highest rock** in the room (or the floor, if there isn't one).

Podemos representar las piuedras com una lista de enteros, suando los bits 1 para indicar donde esta la piedra.

Por ejemplo, para representar:
    
```
.#.
###
.#.
```

Usariamos:
    
```
010
111
010
```
    
Como empiezan dos posiciones desplazados a la izquierda:
    
```
01000
0011100
0001000
```

y rellenando con ceros para obtener el ancho de 7:
    
```
0001000
0011100
0001000
```
    

Es decir, `[8, 28, 8]`. 


In [3]:
# Una funcion para recorrer los bits desde izquierda a derecha
def iter_bits(num: int):
    v = 2 ** (num-1)
    while v > 1:
        yield v
        v = v >> 1
    yield 1

assert list(iter_bits(7)) == [64, 32, 16, 8, 4, 2, 1]

In [4]:
# Un par de constantes para definir el problema

TOP_LEVEL = 0
WIDTH = 7

In [64]:
import copy


class Stone:
    
    def __init__(self, lines: [str], name=''):
        global TOP_LEVEL
        self.name = name
        self.lines = []
        self.width = 0
        self.size = len(lines)
        for line in reversed(lines):
            line = line.replace('#', '1').replace('.', '0')
            self.width = max(self.width, len(line))
            n = int(line, 2)
            self.lines.append(n)
        self.height = TOP_LEVEL + 3
        self.offset = WIDTH - self.width - 2
        for i, value in enumerate(self.lines):
            self.lines[i] = value << self.offset                       
        
    def __repr__(self):
        return f"{self.name or 'Stone'} size: {self.width}x{self.size} height: {self.height}"
        
    def __lshift__(self, places):
        if self.offset + self.width < WIDTH:
            result = copy.copy(self)
            limit = min(places, WIDTH - self.width - self.offset)
            result.lines = [v << limit for v in self.lines]
            result.offset += limit
            return result
        return self
                         
    def __rshift__(self, places):
        if self.offset > 0:
            result = copy.copy(self)
            limit = min(places, self.offset)
            result.lines = [v >> limit for v in self.lines]
            result.offset -= limit
            return result
        return self        
        
    def __str__(self):
        buff = []
        for line in reversed(self.lines):
            sub_buff = [
                '#' if (line & bit_mask) else '.'
                for bit_mask in iter_bits(WIDTH)
            ]
            buff.append(''.join(sub_buff))
        return '\n'.join(buff).strip()
 
    @classmethod
    def minus(cls):
        return cls(['####'], name='minus')
    
    @classmethod
    def plus(cls):
        return cls(['.#.', '###', '.#.'], name='plus')
    
    @classmethod
    def corner(cls):
        return cls(['..#', '..#', '###'], name='corner') 
    
    @classmethod
    def stick(cls):
        return cls(['#', '#', '#', '#'], name='stick')
    
    @classmethod
    def block(cls):
        return cls(['##', '##'], name='block')

In [68]:
TOP_LEVEL = 7
stone = Stone(['####'])
assert stone.height == 10
TOP_LEVEL = 0

In [69]:
minus = Stone.minus()

assert minus.lines == [30,]
assert minus.offset == 1
assert minus.size == 1
assert minus.height == 3
assert str(minus) == '..####.'
print(repr(minus))

minus size: 4x1 height: 3


In [70]:
plus = Stone.plus() 
assert plus.lines == [8, 28, 8]
assert plus.offset == 2
assert plus.size == 3
assert plus.height == 3
assert str(plus) == '...#...\n..###..\n...#...'

In [71]:
corner = Stone.corner()

assert corner.lines == [28, 4, 4]
assert corner.offset == 2
assert corner.size == 3
assert corner.height == 3
assert str(corner) == '....#..\n....#..\n..###..'

In [72]:
stick = Stone.stick()

assert stick.lines == [16, 16, 16, 16]
assert stick.offset == 4
assert stick.size == 4
assert stick.height == 3
assert str(stick) == '..#....\n..#....\n..#....\n..#....'

In [73]:
corner = Stone.block()
assert corner.lines == [24, 24]
assert corner.offset == 3
assert corner.size == 2
assert corner.height == 3
assert str(corner) == '..##...\n..##...'

In [74]:
corner = Stone.corner()
assert corner.offset == 2
corner = corner >> 3
assert corner.offset == 0

In [75]:
block = Stone.block()
assert block.lines == [24, 24]
assert block.offset == 3
assert str(block) == '..##...\n..##...'

In [76]:
#  first the - shape, then the + shape, and so on. [corner, stick, block]
# Once the end of the list is reached, the same order repeats: the 
# `-` shape falls first, sixth, 11th, 16t
import itertools

def rock_sequence():
    yield Stone.minus()
    yield Stone.plus()
    yield Stone.corner()
    yield Stone.stick()
    yield Stone.block()

def rock_falls():
    yield from itertools.cycle(rock_sequence())
    
for i, stone in enumerate(rock_falls()):
    if i == 15:  # Must be a - (Minus)
        print(i, stone, sep='\n')
        break

15
..####.


After a rock appears, it alternates between being pushed by a jet of hot gas one unit (in the direction indicated by the next symbol in the jet pattern) and then falling one unit down. If any movement would cause any part of the rock to move into the walls, floor, or a stopped rock, the movement instead does not occur. If a downward movement would have caused a falling rock to move into the floor or an already-fallen rock, the falling rock stops where it is (having landed on something) and a new rock immediately begins falling.

Drawing falling rocks with @ and stopped rocks with #, the jet pattern in the example above manifests as follows:

The first rock begins falling:

```
|..@@@@.|
|.......|
|.......|
|.......|
+-------+
```

Jet of gas pushes rock right:

```
|...@@@@|
|.......|
|.......|
|.......|
+-------+
```

Rock falls 1 unit:

```
|...@@@@|
|.......|
|.......|
+-------+
```

Jet of gas pushes rock right, but nothing happens:

```
|...@@@@|
|.......|
|.......|
+-------+
```

Rock falls 1 unit:

```
|...@@@@|
|.......|
+-------+
```

Jet of gas pushes rock right, but nothing happens:

```
|...@@@@|
|.......|
+-------+
```

Rock falls 1 unit:

```
|...@@@@|
+-------+
```

Jet of gas pushes rock left:

```
|..@@@@.|
+-------+
```

Rock falls 1 unit, causing it to come to rest:

```
|..####.|
+-------+
```

In [106]:
import collections


def create_stack(size=TOP_LEVEL):
    stack = [0] * (TOP_LEVEL + 3)
    return stack

def as_stack_line(line, symbol='#'):
    buff = [
        symbol if (line & bit_mask) else '.'
        for bit_mask in iter_bits(WIDTH)
        ]
    return ''.join(buff)
    
def print_stack(stack, stone=None):
    if stone:
        index = max(TOP_LEVEL, stone.height + stone.size)
    else:
        index = len(stack)
    if index >= len(stack):
        stack.extend([0] * 10)    
    while index >= 0:
        if stone:
            if stone.height <= index < (stone.height + stone.size):
                buff = []
                stone_subindex = index - stone.height
                for bit_mask in iter_bits(WIDTH):
                    if bit_mask & stack[index]:
                        buff.append('#')
                    elif bit_mask & stone.lines[stone_subindex]:
                        buff.append('@')
                    else:
                        buff.append('.')
                print('|', ''.join(buff), '|', sep='')
            else:
                if index > len(stack):
                    print('|', as_stack_line(0), '|', sep='')    
                else:
                    print('|', as_stack_line(stack[index]), '|', sep='')
        else:
            if index > len(stack):
                print('|', as_stack_line(0), '|', sep='')    
            else:
                print('|', as_stack_line(stack[index]), '|', sep='')            
        index -= 1
    print('└', '-' * WIDTH, '┘', sep='')


def fits(stack, stone):
    results = []
    for index in range(stone.size):
        stack_index = stone.height + index
        if stack_index >= len(stack):
            stack.extend([0] * 10)
        results.append((stone.lines[index] & stack[stack_index]) == 0)
    return all(results)


def move(stack, stone, wind):
    if wind == '>':
        new_stone = copy.copy(stone) >> 1
    else:
        new_stone = copy.copy(stone) << 1
    if fits(stack, new_stone):
        return new_stone
    return stone


def fall(stack, stone):
    new_stone = copy.copy(stone)
    if new_stone.height > 0:
        new_stone.height -= 1
        if fits(stack, new_stone):
            return True, new_stone
    return False, stone
    

def solidify(stack, stone):
    for index in range(stone.size):
        position = stone.height + index
        if position >= len(stack):
            stack.extend([0] * 10)
        stack[position] = stack[position] | stone.lines[index]
    return stack

TOP_LEVEL = 0
stack = create_stack()
stones = [Stone.minus(), Stone.plus()]
winds = air_flow('sample.txt')
for stone in stones:
    stone.height = TOP_LEVEL + 3
    print(f"Stone {stone} begins falling")
    print_stack(stack, stone)
    for wind in winds:
        stone = move(stack, stone, wind)
        print(f"Move {wind}")
        print_stack(stack, stone)
        flag, stone = fall(stack, stone)
        if flag:
            print("Fall")
            print_stack(stack, stone)
            continue   
        _prev = TOP_LEVEL
        ic(TOP_LEVEL, stone.height + stone.size)
        TOP_LEVEL = max(TOP_LEVEL, stone.height + stone.size)
        ic(_prev, TOP_LEVEL)
        stack = solidify(stack, stone)
        print_stack(stack)
        break

ic| TOP_LEVEL: 0, stone.height + stone.size: 1
ic| _prev: 0, TOP_LEVEL: 1
ic| TOP_LEVEL: 1, stone.height + stone.size: 4
ic| _prev: 1, TOP_LEVEL: 4


Stone ..####. begins falling
|.......|
|..@@@@.|
|.......|
|.......|
|.......|
└-------┘
Move >
|.......|
|...@@@@|
|.......|
|.......|
|.......|
└-------┘
Fall
|.......|
|...@@@@|
|.......|
|.......|
└-------┘
Move >
|.......|
|...@@@@|
|.......|
|.......|
└-------┘
Fall
|.......|
|...@@@@|
|.......|
└-------┘
Move >
|.......|
|...@@@@|
|.......|
└-------┘
Fall
|.......|
|...@@@@|
└-------┘
Move <
|.......|
|..@@@@.|
└-------┘
|.......|
|.......|
|.......|
|.......|
|.......|
|.......|
|.......|
|.......|
|.......|
|.......|
|.......|
|.......|
|.......|
|..####.|
└-------┘
Stone ...#...
..###..
...#... begins falling
|.......|
|...@...|
|..@@@..|
|...@...|
|.......|
|.......|
|.......|
|..####.|
└-------┘
Move <
|.......|
|..@....|
|.@@@...|
|..@....|
|.......|
|.......|
|.......|
|..####.|
└-------┘
Fall
|.......|
|..@....|
|.@@@...|
|..@....|
|.......|
|.......|
|..####.|
└-------┘
Move >
|.......|
|...@...|
|..@@@..|
|...@...|
|.......|
|.......|
|..####.|
└-------┘
Fall
|.......|

In [107]:
def air_flow(filename):
    jets = load_input(filename)
    yield from itertools.cycle(jets)

In [169]:
def solution_one(filename, tope=10, tron=False):
    global TOP_LEVEL
    TOP_LEVEL = 0
    stack = create_stack()
    count = 0
    winds = air_flow(filename)
    for stone in rock_falls():
        stone.height = TOP_LEVEL + 3
        count += 1
        if count > tope:
            if tron:
                print_stack(stack, stone)
            return TOP_LEVEL
            break
        for wind in winds:
            if tron:
                ic(count, stone, wind, TOP_LEVEL)
                print_stack(stack, stone)
            stone = move(stack, stone, wind)
            flag, stone = fall(stack, stone)
            if not flag:
                stack = solidify(stack, stone)
                TOP_LEVEL = max(TOP_LEVEL, stone.height + stone.size)
                break
    
 
assert solution_one('sample.txt', 2022) == 3068

In [170]:
sol = solution_one('input.txt', 2022)
print(f"Solution part two: {sol}")

Solution part two: 3209


## Part two

In [183]:
import itertools

def triplewise(iterable):
    # pairwise('ABCDEFG') --> AB BC CD DE EF FG
    a, b, c = itertools.tee(iterable, 3)
    next(b, None)
    next(c, None); next(c, None)
    return zip(a, b, c)


def solidify_two(stack, stone):
    flag = 0
    values = []
    for index in range(stone.size):
        position = stone.height + index
        if position >= len(stack):
            stack.extend([0] * 10)
        value = stack[position] | stone.lines[index]
        values.append(value)
        stack[position] = value
    for i, (a, b, c) in enumerate(triplewise(stack)):
        if a | b | c == 127:
            flag = i
    return flag, stack


def solution_two(filename, tope=10, tron=False):
    global TOP_LEVEL
    spy = 0
    TOP_LEVEL = 0
    prev_count = 0
    stack = create_stack()
    count = 0
    winds = air_flow(filename)
    for stone in rock_falls():
        stone.height = TOP_LEVEL + 3
        count += 1
        if count > tope:
            return prev_count + TOP_LEVEL, spy
            break
        for wind in winds:
            if tron:
                ic(count, stone, wind, TOP_LEVEL)
                print_stack(stack, stone)
            stone = move(stack, stone, wind)
            fallen, stone = fall(stack, stone)
            if not fallen:
                num, stack = solidify_two(stack, stone)
                if num > 0:
                    spy = max(spy, num)
                    stack = stack[num:]
                    prev_count += num
                    TOP_LEVEL = TOP_LEVEL - num
                else:
                    TOP_LEVEL = max(TOP_LEVEL, stone.height + stone.size)
                break
                
    
 
%time solution_two('input.txt', 2022)

CPU times: user 159 ms, sys: 0 ns, total: 159 ms
Wall time: 159 ms


(3179, 154)

In [180]:
%time solution_one('input.txt', 2022)

CPU times: user 142 ms, sys: 0 ns, total: 142 ms
Wall time: 144 ms


3209

A new rock begins falling:

```
|...@...|
|..@@@..|
|...@...|
|.......|
|.......|
|.......|
|..####.|
+-------+
```

Jet of gas pushes rock left:

```
|..@....|
|.@@@...|
|..@....|
|.......|
|.......|
|.......|
|..####.|
+-------+
```

Rock falls 1 unit:

```
|..@....|
|.@@@...|
|..@....|
|.......|
|.......|
|..####.|
+-------+
```

Jet of gas pushes rock right:

```
|...@...|
|..@@@..|
|...@...|
|.......|
|.......|
|..####.|
+-------+
```

Rock falls 1 unit:

```
|...@...|
|..@@@..|
|...@...|
|.......|
|..####.|
+-------+
```

Jet of gas pushes rock left:

```
|..@....|
|.@@@...|
|..@....|
|.......|
|..####.|
+-------+
```

Rock falls 1 unit:

```
|..@....|
|.@@@...|
|..@....|
|..####.|
+-------+
```

Jet of gas pushes rock right:

```
|...@...|
|..@@@..|
|...@...|
|..####.|
+-------+
```

Rock falls 1 unit, causing it to come to rest:

```
|...#...|
|..###..|
|...#...|
|..####.|
+-------+
```

A new rock begins falling:

```
|....@..|
|....@..|
|..@@@..|
|.......|
|.......|
|.......|
|...#...|
|..###..|
|...#...|
|..####.|
+-------+
```

The moment each of the next few rocks begins falling, you would see this:

```
|..@....|
|..@....|
|..@....|
|..@....|
|.......|
|.......|
|.......|
|..#....|
|..#....|
|####...|
|..###..|
|...#...|
|..####.|
+-------+

|..@@...|
|..@@...|
|.......|
|.......|
|.......|
|....#..|
|..#.#..|
|..#.#..|
|#####..|
|..###..|
|...#...|
|..####.|
+-------+

|..@@@@.|
|.......|
|.......|
|.......|
|....##.|
|....##.|
|....#..|
|..#.#..|
|..#.#..|
|#####..|
|..###..|
|...#...|
|..####.|
+-------+

|...@...|
|..@@@..|
|...@...|
|.......|
|.......|
|.......|
|.####..|
|....##.|
|....##.|
|....#..|
|..#.#..|
|..#.#..|
|#####..|
|..###..|
|...#...|
|..####.|
+-------+

|....@..|
|....@..|
|..@@@..|
|.......|
|.......|
|.......|
|..#....|
|.###...|
|..#....|
|.####..|
|....##.|
|....##.|
|....#..|
|..#.#..|
|..#.#..|
|#####..|
|..###..|
|...#...|
|..####.|
+-------+

|..@....|
|..@....|
|..@....|
|..@....|
|.......|
|.......|
|.......|
|.....#.|
|.....#.|
|..####.|
|.###...|
|..#....|
|.####..|
|....##.|
|....##.|
|....#..|
|..#.#..|
|..#.#..|
|#####..|
|..###..|
|...#...|
|..####.|
+-------+

|..@@...|
|..@@...|
|.......|
|.......|
|.......|
|....#..|
|....#..|
|....##.|
|....##.|
|..####.|
|.###...|
|..#....|
|.####..|
|....##.|
|....##.|
|....#..|
|..#.#..|
|..#.#..|
|#####..|
|..###..|
|...#...|
|..####.|
+-------+

|..@@@@.|
|.......|
|.......|
|.......|
|....#..|
|....#..|
|....##.|
|##..##.|
|######.|
|.###...|
|..#....|
|.####..|
|....##.|
|....##.|
|....#..|
|..#.#..|
|..#.#..|
|#####..|
|..###..|
|...#...|
|..####.|
+-------+
```

To prove to the elephants your simulation is accurate, they want to know how tall the tower will get after 2022 rocks have stopped (but before the 2023rd rock begins falling). In this example, the tower of rocks will be **3068** units tall.

How many units tall will the tower of rocks be after 2022 rocks have stopped falling?

In [ ]:
sol = solution_one('input.txt')
print(f"Solution part one: {sol}")